In [11]:
!pip install beautifulsoup4

     |████████████████████████████████| 115 kB 964 kB/s eta 0:00:01


In [10]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

# 데이터 전처리
## 수집한 우리나라 꽃 21종 이미지로 train, val data 생성

## 2.1 Flower list 생성

In [77]:
flowers = [['수련','장미','해바라기','연꽃','카네이션','수선화','목련','동백','민들레','데이지','무궁화','진달래','모란','코스모스','백합','할미꽃','나팔꽃','찔레꽃','수국','은방울꽃','프리지아'],['waterLily','rose','sunflower','lotus','carnation','daffodill','magnolia','camellia','dandelion','daisy','mugungwha','amalea','moran','cosmos','lily','pasqueflower','morning glory','wild rose','hydrangea','droplet','freesia']]
flowers_kor = flowers[0]
flowers_eng = flowers[1]
flowers = []
for flower_kor, flower_eng in zip(flowers_kor, flowers_eng):
    flowers.append([flower_kor,flower_eng])
pd.DataFrame(flowers, columns=['꽃 이름','영문 이름'])

,꽃 이름,영문 이름
0,수련,waterLily
1,장미,rose
2,해바라기,sunflower
3,연꽃,lotus
4,카네이션,carnation
5,수선화,daffodill
6,목련,magnolia
7,동백,camellia
8,민들레,dandelion
9,데이지,daisy


## 2.2 전체 데이터 dataframe으로 저장

In [90]:
def xml_txt_to_df(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = ('data/imgs/'+member[0].text+'/'+root.find('filename').text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text),
                     member[0].text
                     )
            xml_list.append(value)
    for flower in flowers_eng:
        for xml_file in glob.glob(path + '/' + flower + '/*.xml'):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for member in root.findall('object'):
                value = ('data/imgs/'+flower+'/'+root.find('filename').text,
                         int(member[4][0].text),
                         int(member[4][1].text),
                         int(member[4][2].text),
                         int(member[4][3].text),
                         member[0].text
                         )
                xml_list.append(value)

    column_name = ['filename','x1', 'y1', 'x2', 'y2', 'classname']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

### 전체 데이터 확인

In [91]:
df = xml_txt_to_df(image_path)
df

,filename,x1,y1,x2,y2,classname
0,data/imgs/daisy/image_0807.jpg,40,61,585,484,daisy
1,data/imgs/daisy/image_0813.jpg,194,9,697,500,daisy
2,data/imgs/magnolia/image_05462.jpg,45,28,613,456,magnolia
3,data/imgs/rose/image_01162.jpg,172,40,479,363,rose
4,data/imgs/rose/image_01176.jpg,167,115,376,424,rose
...,...,...,...,...,...,...
4465,data/imgs/freesia/naver_0468.jpg,17,93,272,378,freesia
4466,data/imgs/freesia/naver_0253.jpg,40,110,150,248,freesia
4467,data/imgs/freesia/naver_0253.jpg,144,149,248,272,freesia
4468,data/imgs/freesia/naver_0253.jpg,5,211,83,304,freesia


### 클래스별 데이터 개수 확인

In [96]:
flowers = ['waterLily','rose','sunflower','lotus','carnation','daffodill','magnolia','camellia','dandelion','daisy','mugungwha','amalea','moran','cosmos','lily','pasqueflower','morning glory','wild rose','hydrangea','droplet','freesia']

for flower in flowers:
    str_exp = "classname=='{}'".format(flower)
    print(flower, len(df.query(str_exp)))
    

waterLily 115
rose 92
sunflower 74
lotus 112
carnation 75
daffodill 88
magnolia 111
camellia 95
dandelion 167
daisy 124
mugungwha 214
amalea 242
moran 149
cosmos 365
lily 243
pasqueflower 467
morning glory 271
wild rose 468
hydrangea 451
droplet 448
freesia 99


## 2.3 train, val dataset으로 분리

In [95]:
dd[10:]

,filename,x1,y1,x2,y2,classname
57,data/imgs/image_01177.jpg,5,1,709,500,rose
60,data/imgs/image_01163.jpg,109,71,471,349,rose
68,data/imgs/image_01175.jpg,127,1,600,461,rose
69,data/imgs/image_01161.jpg,130,46,531,438,rose
72,data/imgs/image_01149.jpg,69,40,565,500,rose
...,...,...,...,...,...,...
1025,data/imgs/image_01219.jpg,6,88,479,600,rose
1037,data/imgs/image_01190.jpg,102,22,623,500,rose
1038,data/imgs/image_01184.jpg,75,69,425,407,rose
1041,data/imgs/image_01153.jpg,136,1,548,487,rose


In [114]:
flowers = ['waterLily','rose','sunflower','lotus','carnation','daffodill','magnolia','camellia','dandelion','daisy','mugungwha','amalea','moran','cosmos','lily','pasqueflower','morning glory','wild rose','hydrangea','droplet','freesia']
train = pd.DataFrame()
val = pd.DataFrame()
test = pd.DataFrame()            
for flower in flowers:
    str_exp = "classname=='{}'".format(flower)
    train = train.append(df.query(str_exp)[15:])
    val = val.append(df.query(str_exp)[:10])
    test = test.append(df.query(str_exp)[10:15])


In [115]:
train

,filename,x1,y1,x2,y2,classname
75,data/imgs/waterLily/image_00294.jpg,114,5,588,487,waterLily
76,data/imgs/waterLily/image_00280.jpg,72,59,345,308,waterLily
77,data/imgs/waterLily/image_00280.jpg,292,82,594,344,waterLily
80,data/imgs/waterLily/image_00323.jpg,141,103,506,435,waterLily
81,data/imgs/waterLily/image_00337.jpg,80,32,523,462,waterLily
...,...,...,...,...,...,...
4465,data/imgs/freesia/naver_0468.jpg,17,93,272,378,freesia
4466,data/imgs/freesia/naver_0253.jpg,40,110,150,248,freesia
4467,data/imgs/freesia/naver_0253.jpg,144,149,248,272,freesia
4468,data/imgs/freesia/naver_0253.jpg,5,211,83,304,freesia


In [116]:
val

,filename,x1,y1,x2,y2,classname
5,data/imgs/waterLily/image_00268.jpg,100,33,617,460,waterLily
7,data/imgs/waterLily/image_00254.jpg,37,45,610,460,waterLily
8,data/imgs/waterLily/image_00283.jpg,424,60,789,367,waterLily
9,data/imgs/waterLily/image_00297.jpg,66,10,601,374,waterLily
16,data/imgs/waterLily/image_00308.jpg,57,129,489,457,waterLily
...,...,...,...,...,...,...
4376,data/imgs/freesia/naver_0538.jpg,90,23,335,187,freesia
4377,data/imgs/freesia/naver_0665.jpg,193,113,439,350,freesia
4378,data/imgs/freesia/naver_0843.jpg,176,6,513,373,freesia
4379,data/imgs/freesia/naver_0843.jpg,31,104,280,400,freesia


In [117]:
test

,filename,x1,y1,x2,y2,classname
51,data/imgs/waterLily/image_00296.jpg,151,123,613,365,waterLily
54,data/imgs/waterLily/image_00282.jpg,83,29,476,458,waterLily
55,data/imgs/waterLily/image_00255.jpg,68,5,608,455,waterLily
56,data/imgs/waterLily/image_00269.jpg,123,74,526,442,waterLily
73,data/imgs/waterLily/image_00257.jpg,121,86,578,358,waterLily
...,...,...,...,...,...,...
4381,data/imgs/freesia/naver_0099.jpg,203,50,534,361,freesia
4382,data/imgs/freesia/naver_0338.jpg,31,7,229,197,freesia
4383,data/imgs/freesia/naver_0338.jpg,167,75,363,288,freesia
4384,data/imgs/freesia/naver_0338.jpg,12,185,217,394,freesia


### 저장

In [119]:
val.to_csv('21class/val.csv', index=None)

In [120]:
train.to_csv('21class/train.csv', index=None)

In [121]:
test.to_csv('21class/test.csv', index=None)